In [ ]:
!wget https://noto-website-2.storage.googleapis.com/pkgs/NotoSans-hinted.zip

--2021-03-28 14:53:00--  https://noto-website-2.storage.googleapis.com/pkgs/NotoSans-hinted.zip
Resolving noto-website-2.storage.googleapis.com (noto-website-2.storage.googleapis.com)... 74.125.20.128, 2607:f8b0:400e:c07::80
Connecting to noto-website-2.storage.googleapis.com (noto-website-2.storage.googleapis.com)|74.125.20.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16796031 (16M) [application/zip]
Saving to: ‘NotoSans-hinted.zip’

NotoSans-hinted.zip 100%[===================>]  16.02M  --.-KB/s    in 0.1s    

2021-03-28 14:53:00 (116 MB/s) - ‘NotoSans-hinted.zip’ saved [16796031/16796031]



In [ ]:
!unzip "NotoSans-hinted.zip"

Archive:  NotoSans-hinted.zip
  inflating: LICENSE_OFL.txt         
  inflating: NotoSans-Black.ttf      
  inflating: NotoSans-BlackItalic.ttf  
  inflating: NotoSans-Bold.ttf       
  inflating: NotoSans-BoldItalic.ttf  
  inflating: NotoSans-Condensed.ttf  
  inflating: NotoSans-CondensedBlack.ttf  
  inflating: NotoSans-CondensedBlackItalic.ttf  
  inflating: NotoSans-CondensedBold.ttf  
  inflating: NotoSans-CondensedBoldItalic.ttf  
  inflating: NotoSans-CondensedExtraBold.ttf  
  inflating: NotoSans-CondensedExtraBoldItalic.ttf  
  inflating: NotoSans-CondensedExtraLight.ttf  
  inflating: NotoSans-CondensedExtraLightItalic.ttf  
  inflating: NotoSans-CondensedItalic.ttf  
  inflating: NotoSans-CondensedLight.ttf  
  inflating: NotoSans-CondensedLightItalic.ttf  
  inflating: NotoSans-CondensedMedium.ttf  
  inflating: NotoSans-CondensedMediumItalic.ttf  
  inflating: NotoSans-CondensedSemiBold.ttf  
  inflating: NotoSans-CondensedSemiBoldItalic.ttf  
  inflating: NotoSans-Conde

In [ ]:
from bs4 import BeautifulSoup
from PIL import Image, ImageDraw, ImageFont
import requests
data = []
def crawNewsData(baseUrl, url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")
    titles = soup.findAll('h3', class_='title-news')
    links = [link.find('a').attrs["href"] for link in titles]
    
    for link in links:
        news = requests.get(baseUrl + link)
        soup = BeautifulSoup(news.content, "html.parser")
        title = soup.find("h1", class_="article-title").text
        abstract = soup.find("h2", class_="sapo").text
        body = soup.find("div", id="main-detail-body")
        content = ""
        try:
            content = body.findChildren("p", recursive=False)[0].text + body.findChildren("p", recursive=False)[1].text
        except:
            content = ""
        image = body.find("img").attrs["src"]
        data.append({
            "title": title,
            "abstract": abstract,
            "content": content,
            "image": image,
        })
        print("craw " + title)
    return data


def writeToImage(image, text, position, font, color, maxLine):
    charPerLine = 650 // font.getsize('x')[0]
    pen = ImageDraw.Draw(image)
    yStart = position[1]
    xStart = position[0]
    point = 0
    prePoint = 0
    while point < len(text):
        prePoint = point
        point += charPerLine
        while point < len(text) and text[point] != " ":
            point -= 1
        pen.text((xStart, yStart), text[prePoint:point], font=font, fill=color)
        yStart += font.getsize('hg')[1]
        maxLine -= 1
        if (maxLine == 0):
            if (point < len(text)):
                pen.text((xStart, yStart), "...", font=font, fill="black")
            break


def makeFastNews(data):
    for index, item in enumerate(data):
        # make new image and tool to draw
        image = Image.new('RGB', (650, 750), color="white")
        pen = ImageDraw.Draw(image)
        # load image from internet => resize => paste to main image
        pen.rectangle(((0, 0), (650, 300)), fill="grey")
        newsImage = Image.open(requests.get(item["image"], stream=True).raw)
        newsImage.thumbnail((650, 300), Image.ANTIALIAS)
        image.paste(newsImage, (650 // 2 - newsImage.width // 2, 300 // 2 - newsImage.height//2))
        ## write title
        titleFont = ImageFont.truetype("/content/NotoSans-Light.ttf", 22)
        writeToImage(image, item["title"], (10, 310), titleFont, "black", 3)
        abstractFont = ImageFont.truetype("/content/NotoSans-Light.ttf", 15)
        writeToImage(image, item["abstract"], (10, 390), abstractFont, "gray", 4)
        contentFont = ImageFont.truetype("/content/NotoSans-Light.ttf", 20)
        writeToImage(image, item["content"], (10, 460), contentFont, "black", 11)
        name = "news-" + str(index) + ".png"
        image.save("news/" + name)
        print("saved to " + "news/" + name)


if __name__ == "__main__":
    crawNewsData("https://tuoitre.vn", "https://tuoitre.vn/phap-luat.htm")
    crawNewsData("https://tuoitre.vn", "https://tuoitre.vn/kinh-doanh.htm")
    crawNewsData("https://tuoitre.vn", "https://tuoitre.vn/giai-tri.htm")
    # crawNewsData("https://tuoitre.vn", "https://tuoitre.vn/khoa-hoc.htm")
    # crawNewsData("https://tuoitre.vn", "https://tuoitre.vn/the-gioi.htm")
    makeFastNews(data)

craw Công an Bình Chánh xác nhận 2 CSGT Bình Chánh do thiếu kiềm chế nên đã đánh 2 thanh niên
craw Đang ăn cưới bỗng cãi nhau, hẹn ra ngoài 'nói chuyện' bằng súng, một người chết
craw Đường mới làm ở Hóc Môn, chiều 28-3, quái xế đổ đến 'đông như quân Nguyên'
craw Vụ đầu bếp chết trong quán nhậu, chồng cũ của chủ quán qua cơn nguy kịch
craw Tổng giám đốc ‘vẽ dự án ma’ bán đất, Công an Hà Nội vào cuộc
craw 'Trốn' án dân sự, dính án hình sự
craw Vụ 'nam thanh niên bị chôn sống’: Nhân vật chính nói quay vui để cảnh báo
craw Dàn cựu lãnh đạo VEAM bị đề nghị truy tố vì gây thiệt hại hàng trăm tỉ
craw 19 bị cáo gây thiệt hại lớn ở dự án Gang thép Thái Nguyên sắp hầu tòa
craw Đầu bếp chết trong quán nhậu, nghi bị chủ quán đâm vì ghen tuông
craw Xác minh video nam thanh niên bị đánh đập rồi trùm đầu ‘chôn sống’
craw Khởi tố vụ tổ chức đưa người Trung Quốc nhập cảnh Việt Nam trái phép
craw ‘Yêu’ bé gái 15 tuổi quen qua mạng, thanh niên bị khởi tố
craw Thanh niên dùng clip sex tống tiền 200 triệu